In [1]:
import pandas as pd
from tqdm import tqdm
from collections import Counter
import re

# we can not find 'PanGU-Σ', 'Luminous' in close Source index, cannot find 'Galatica', 'YaLM', 'PanGu-α' in open source list , so delete them.
openSourceList = ['T5', 'mT5', 'CPM-2','T0','GPT-NeoX-20B','CodeGen','Tk-Instruct','UL2','OPT','NLLB','BLOOM','GLM','Flan-T5','mT0','BLOOMZ','OPT-IML','Pythia','LLaMA','Vicuna','ChatGLM','CodeGeeX','Koala']
closeSourceList = ['GShard','GPT-3','GPT 3','LaMDA','HyperCLOVA','Codex','ERNIE 3','Jurassic-1','FLAN','MT-NLG','Yuan 1.0','Anthropic','WebGPT','Gopher','ERNIE 3.0 Titan','GLaM','InstructGPT','ChatGPT','AlphaCode','Chinchilla','PaLM','Cohere','AlexaTM','Sparrow','WeLM','U-PaLM','Flan-PaLM','Flan-U-PaLM','Alpaca','GPT-4' ,'GPT 4','Claude']

readfile = "/data/jx4237data/DataForChatGPTinnovationWaves/"
df = pd.read_csv(readfile + 'LLMs0819.csv',dtype=object)
# some paper we need to delete:
# “fake” palm paper 1909.02134
# fake openai 1506.04006
delete_list = ['1506.04006','1909.02134']
df = df[~df['id'].isin(delete_list)]
# model2paper
model2paper = {}
for model in tqdm(openSourceList+closeSourceList):
    if model in model2paper:
        print('error')
    model2paper[model] = set()
    for paper in df[df['abstract'].str.contains("\\b%s\\b"%model)  | df['title'].str.contains("\\b%s\\b"%model)]['id']:
        model2paper[model].add(paper)

# paper2model
paper2model = {}
for model, paperSet in model2paper.items():
    for paper in paperSet:
        if paper not in paper2model:
            paper2model[paper] =[model]
        else:
            paper2model[paper].append(model)


openPaperSet = set()
closePaperSet = set()
for k,v in model2paper.items():
    if k in closeSourceList:
        for paper in v:
            closePaperSet.add(paper)
    if k in openSourceList:
        for paper in v:
            openPaperSet.add(paper)
pureOpen = openPaperSet - closePaperSet
pureClose = closePaperSet - openPaperSet
mixed = openPaperSet & closePaperSet
all = openPaperSet | closePaperSet

paper2pubdate = {}
for i in range(len(df)):
    paper2pubdate[list(df['id'])[i]] = list(df['update_date'])[i]

100%|██████████| 54/54 [00:06<00:00,  7.88it/s]


In [47]:
with open('stopwords.txt') as f:
    stopwordslist = f.read().splitlines()
stopwordslist.append('')

In [57]:
def genrawdata(modelname):
    c = Counter()
    for i in df[df['id'].isin(model2paper[modelname])]['abstract'] + df[df['id'].isin(model2paper[modelname])]['title']:
        for word in re.split('[, \.!?:()]+', i.strip()):
            if word.lower() in stopwordslist:
                continue
            else:
                c.update([word.lower()])
    totalunique = len(c)
    sumcounts = sum(c.values())
    model_list = []
    for k,v in sorted(c.items(), key= lambda x:-x[1]):
        model_list.append({
            'types':k,
            'counts':v,
            'totalunique':totalunique,
            'probs':v/sumcounts,
        })
    return {modelname:model_list}

In [71]:
def genrawdataByname(paperset,name):
    c = Counter()
    for i in df[df['id'].isin(paperset)]['abstract'] + df[df['id'].isin(paperset)]['title']:
        for word in re.split('[, \.!?:()]+', i.strip()):
            if word.lower() in stopwordslist:
                continue
            else:
                c.update([word.lower()])
    totalunique = len(c)
    sumcounts = sum(c.values())
    model_list = []
    for k,v in sorted(c.items(), key= lambda x:-x[1]):
        model_list.append({
            'types':k,
            'counts':v,
            'totalunique':totalunique,
            'probs':v/sumcounts,
        })
    return {name:model_list}

In [72]:
model2paper

{'T5': {'1910.02054',
  '1911.03705',
  '2004.01909',
  '2004.03607',
  '2005.10433',
  '2007.06225',
  '2007.08426',
  '2008.05221',
  '2008.08769',
  '2010.02600',
  '2010.09697',
  '2011.00696',
  '2011.07956',
  '2012.00955',
  '2101.00063',
  '2101.00416',
  '2101.03961',
  '2102.02779',
  '2102.09094',
  '2102.12206',
  '2103.04350',
  '2103.10360',
  '2103.16063',
  '2104.08145',
  '2104.08620',
  '2104.08691',
  '2104.10350',
  '2105.13626',
  '2105.14897',
  '2106.00737',
  '2106.02171',
  '2106.04571',
  '2106.04718',
  '2106.06899',
  '2106.15332',
  '2107.02137',
  '2107.07261',
  '2107.08146',
  '2108.04026',
  '2108.05857',
  '2108.08111',
  '2109.02593',
  '2109.04645',
  '2109.04672',
  '2109.05093',
  '2109.07377',
  '2109.07684',
  '2109.07958',
  '2109.08079',
  '2109.08668',
  '2109.10686',
  '2110.00672',
  '2110.01509',
  '2110.01552',
  '2110.02370',
  '2110.08525',
  '2110.08529',
  '2111.00310',
  '2111.07658',
  '2111.15622',
  '2112.00791',
  '2112.04630',
  

In [73]:
import json
opensorcePaper = set()
closedsorcePaper = set()
openSourceList
closeSourceList
for i in openSourceList:
    opensorcePaper = opensorcePaper | model2paper[i]
for i in closeSourceList:
    closedsorcePaper = closedsorcePaper | model2paper[i]

s = json.dumps({**genrawdataByname(opensorcePaper,'open sorce'), **genrawdataByname(closedsorcePaper,'closed sorce'),**genrawdata('LLaMA'),**genrawdata('GPT-4'),**genrawdata('GPT-3'),**genrawdata('T5'),**genrawdata('PaLM'), **genrawdata('ChatGPT'), })
open("LLMs.json","w").write(s)

6546875